In [2]:
import math  # Just ignore this :-)

def log(x):
    if x == 0:
        return float('-inf')
    return math.log(x)

# CTiB E2023 - Week 11 - Exercises

# Theoretical exercises

***Exercise 1***: How many terms are there in the sum on slide 37 from the lecture on Nov 13 for computing $P({\bf X}|\Theta)$? Why?



<span style="color:magenta">Når man skal finde summen af sandsynligheder for en bestemt HMM, starter man først i Z1. Det tager 1 at finde den værdi (fra inital prob), for at finde x1, skal man bruge z1 og hente værdien af emission prob, det tager altså også 1.
Når man skal finde alle efterfølgende z afhænger de at z før - tager altså også 1. Det samme gælde efterfølgende x, hvor det afhænger af den tilsvarende z.
term = hver gang man skal hente en sandsynlighed. 
Altså er summen af terms på slide 37 N*2 (N = længden af den observeret sekvens og x2 fordi der både skal bruges transition og emission)
Fra gennemgang: K^N. 
</span>

![[joint prob.png]]

***Exercise 2***: How many terms are there in the maximization on slide 4 from the lecure on Nov 20 for computing the Viterbi decoding ${\bf Z}^*$? Why?



<span style="color:magenta">
Viterbi decoding: Man prøver at finde Z imellem alle Z (Z=K^N) der maximerer joint probability. 

Vi har N*K matrix, hvor N er længden af de observeret states og K er antallet af hidden state, betydende at k er antallet af rækker.
Når vi skal udfylde den første kolonne i vores matrix, skal vi gange initial prob og emission prob sammen - det tager altså 2 terms at udfylde et felt i vores matrix => det tager k*2 terms at udfylde den første kolonne.
De efterfølgende kolonner: Så for at udfylde et felt skal man gange sandsynligheden for at være i det state efter det forrig state, det udregnes ved at gange emission og transition prob. Man får da k*2. For hver pil tilføjes k*2 terms og max tager 1 term, vi får da k*2+1*k. 

Det vil altså tage $2\cdot k+(N-1)\cdot(k\cdot(k\cdot 2+1))$

(N-1 fordi den første kolonne udregnes anderledes end resten)
</span>

Fra gennemgang: K^N

***Exercise 3***: Where in the derivation of $\omega({\bf z}_n$) on slide 8 do we use that the fact that we are working with hidden Markov models? And how do we use it?

Fra step 1 til step 2, hvor man begynder at dele udtrykket op, så man kan skrive hvordan ting afhænger af hinanden. 

Exercise 4:
Fra gennemgang: vi skal gå igennem hver værdi af x i x1-xN, så ?

Exercise 5:

Exercise 6:
K*2 fordi man kun skal gemme to kolonner ad gangen for at udfylde tabellen da n kun afhænger af den forrige kolonne.

# Practical exercises

You are given the same 7-state HMM and helper functions that you used last week:

In [3]:
class hmm:
    def __init__(self, init_probs, trans_probs, emission_probs):
        self.init_probs = init_probs
        self.trans_probs = trans_probs
        self.emission_probs = emission_probs

In [4]:
init_probs_7_state = [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00]

trans_probs_7_state = [
    [0.00, 0.00, 0.90, 0.10, 0.00, 0.00, 0.00],
    [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.05, 0.90, 0.05, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
    [0.00, 0.00, 0.00, 0.10, 0.90, 0.00, 0.00],
]

emission_probs_7_state = [
    #   A     C     G     T
    [0.30, 0.25, 0.25, 0.20],
    [0.20, 0.35, 0.15, 0.30],
    [0.40, 0.15, 0.20, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.20, 0.40, 0.30, 0.10],
    [0.30, 0.20, 0.30, 0.20],
    [0.15, 0.30, 0.20, 0.35],
]

hmm_7_state = hmm(init_probs_7_state, trans_probs_7_state, emission_probs_7_state)

In [5]:
def translate_observations_to_indices(obs):
    mapping = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    return [mapping[symbol.lower()] for symbol in obs]

def translate_indices_to_observations(indices):
    mapping = ['a', 'c', 'g', 't']
    return ''.join(mapping[idx] for idx in indices)

def translate_path_to_indices(path):
    return list(map(lambda x: int(x), path))

def translate_indices_to_path(indices):
    return ''.join([str(i) for i in indices])

# 1 - Viterbi Decoding

Below you will implement and experiment with the Viterbi algorithm. The implementation has been split into three parts:

1. Fill out the $\omega$ table using the recursion presented at the lecture.
2. Find the state with the highest probability after observing the entire sequence of observations.
3. Backtrack from the state found in the previous step to obtain the optimal path.

We'll be working with the 7-state model (`hmm_7_state`) and the helper function for translating between observations, hidden states, and indicies, as introduced above (and also used last week).

Additionally, you're given the function below that constructs a table of a specific size filled with zeros.

In [6]:
def make_table(m, n):
    """Make a table with `m` rows and `n` columns filled with zeros."""
    return [[0] * n for _ in range(m)]

You'll be testing your code with the same two sequences as last week, i.e:

In [7]:
x_short = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'
z_short = '33333333333321021021021021021021021021021021021021'

In [8]:
x_long = 'TGAGTATCACTTAGGTCTATGTCTAGTCGTCTTTCGTAATGTTTGGTCTTGTCACCAGTTATCCTATGGCGCTCCGAGTCTGGTTCTCGAAATAAGCATCCCCGCCCAAGTCATGCACCCGTTTGTGTTCTTCGCCGACTTGAGCGACTTAATGAGGATGCCACTCGTCACCATCTTGAACATGCCACCAACGAGGTTGCCGCCGTCCATTATAACTACAACCTAGACAATTTTCGCTTTAGGTCCATTCACTAGGCCGAAATCCGCTGGAGTAAGCACAAAGCTCGTATAGGCAAAACCGACTCCATGAGTCTGCCTCCCGACCATTCCCATCAAAATACGCTATCAATACTAAAAAAATGACGGTTCAGCCTCACCCGGATGCTCGAGACAGCACACGGACATGATAGCGAACGTGACCAGTGTAGTGGCCCAGGGGAACCGCCGCGCCATTTTGTTCATGGCCCCGCTGCCGAATATTTCGATCCCAGCTAGAGTAATGACCTGTAGCTTAAACCCACTTTTGGCCCAAACTAGAGCAACAATCGGAATGGCTGAAGTGAATGCCGGCATGCCCTCAGCTCTAAGCGCCTCGATCGCAGTAATGACCGTCTTAACATTAGCTCTCAACGCTATGCAGTGGCTTTGGTGTCGCTTACTACCAGTTCCGAACGTCTCGGGGGTCTTGATGCAGCGCACCACGATGCCAAGCCACGCTGAATCGGGCAGCCAGCAGGATCGTTACAGTCGAGCCCACGGCAATGCGAGCCGTCACGTTGCCGAATATGCACTGCGGGACTACGGACGCAGGGCCGCCAACCATCTGGTTGACGATAGCCAAACACGGTCCAGAGGTGCCCCATCTCGGTTATTTGGATCGTAATTTTTGTGAAGAACACTGCAAACGCAAGTGGCTTTCCAGACTTTACGACTATGTGCCATCATTTAAGGCTACGACCCGGCTTTTAAGACCCCCACCACTAAATAGAGGTACATCTGA'
z_long = '3333321021021021021021021021021021021021021021021021021021021021021021033333333334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210321021021021021021021021033334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563333333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456332102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102103210210210210210210210210210210210210210210210210210210210210210'

Remember to translate these sequences to indices before using them with your algorithms.

## Implementing without log-transformation

First, we will implement the algorithm without log-transformation. This will cause issues with numerical stability (like above when computing the joint probability), so we will use the log-transformation trick to fix this in the next section.

### Computation of the $\omega$ table

In [11]:
def compute_w(model, x):
    k = len(model.init_probs)
    n = len(x)
    x = translate_observations_to_indices(x)
    
    w = make_table(k, n)
    #print(w)
    # Base case: fill out w[i][0] for i = 0..k-1
    for j in range(k):
        w[j][0] = model.init_probs[j]*model.emission_probs[j][x[0]]
    # ...
    
    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    for i in range(1, n): #kolonnerne
        for j in range(k): #rækkerne #den k vi er i
            max_prob = 0
            for l in range(k): #den k vi kommer fra
                prob = w[l][i-1]*model.trans_probs[l][j]*model.emission_probs[j][x[i]] #den øverste pil
                if prob > max_prob:
                    max_prob = prob
            w[j][i] = max_prob
    return w
                
print(compute_w(hmm_7_state, 'GTTTAGC'))            


[[0.0, 0, 0, 0.0001875, 6.328125000000001e-05, 7.910156250000001e-06, 2.53125e-06], [0.0, 0, 0.0009375, 0.00021093750000000002, 3.1640625000000005e-05, 1.0125e-05, 3.986718750000001e-06], [0.0, 0.003125, 0.0007031250000000001, 0.00015820312500000003, 6.75e-05, 1.1390625000000002e-05, 1.0812194824218752e-06], [0.25, 0.05625, 0.01265625, 0.0028476562500000004, 0.0006407226562500001, 0.00014416259765625004, 3.243658447265626e-05], [0.0, 0.0012500000000000002, 0.00028125000000000003, 6.328125000000001e-05, 2.8476562500000005e-05, 9.610839843750001e-06, 2.883251953125001e-06], [0.0, 0, 0.00025000000000000006, 5.625000000000001e-05, 1.8984375000000002e-05, 8.54296875e-06, 1.9221679687500005e-06], [0.0, 0, 0, 8.750000000000001e-05, 8.437500000000002e-06, 3.796875000000001e-06, 2.5628906250000003e-06]]


### Finding the joint probability of an optimal path

Now, write a function that given the $\omega$-table, returns the probability of an optimal path through the HMM. As explained in the lecture, this corresponds to finding the highest probability in the last column of the table.

In [15]:
def opt_path_prob(w):
    max_prob = 0
    for j in range(len(w)):
        prob = w[j][-1]
        if prob > max_prob:
            max_prob = prob
    return max_prob

table_test=compute_w(hmm_7_state, 'GTTTAGC')
print(opt_path_prob(table_test))   


3.243658447265626e-05


Now test your implementation in the box below:

In [16]:
w = compute_w(hmm_7_state, x_short)
opt_path_prob(w)

1.9114255184318858e-31

Now do the same for `x_long`. What happens?

In [17]:
w = compute_w(hmm_7_state, x_long)
opt_path_prob(w)

0

### Obtaining an optimal path through backtracking

Implement backtracking to find a most probable path of hidden states given the $\omega$-table.

In [27]:
def backtrack(model, x, w):
    backtrack_list = []
    x = translate_observations_to_indices(x)
    
    max_prob = 0
    max_prob_position = None 
    for j in range(len(w)):
        prob = w[j][-1]
        if prob > max_prob:
            max_prob = prob
            max_prob_position = j
    backtrack_list.append(max_prob_position)

    pre_prob = max_prob
    pre_prob_position = max_prob_position
    
    for i in range(-2, -len(x), -1):
        for j in range(len(w)):
            prob = w[j][i]*model.trans_probs[j][pre_prob_position]*model.emission_probs[pre_prob_position][x[i+1]]
            if math.isclose(prob,  pre_prob):
                pre_prob = w[j][i]
                pre_prob_position = j
                backtrack_list.append(pre_prob_position)

    return backtrack_list

In [28]:
w = compute_w(hmm_7_state, x_short)
z_viterbi = backtrack(hmm_7_state, x_short, w)
print(z_viterbi)

[1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


Now do the same for `x_long`. What happens?

In [ ]:
# Your code here ...

## Implementing with log-transformation

Now implement the Viterbi algorithm with log-transformation. The steps are the same as above.

### Computation of the (log-transformed) $\omega$ table

In [ ]:
def compute_w_log(model, x):
    k = len(model.init_probs)
    n = len(x)
    
    w = make_table(k, n)
    
    # Base case: fill out w[i][0] for i = 0..k-1
    # ...
    
    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    # ...

In [ ]:
import math  # Just ignore this :-)

def log(x):
    if x == 0:
        return float('-inf')
    return math.log(x)

### Finding the (log-transformed) joint probability of an optimal path

In [ ]:
def opt_path_prob_log(w):
    pass

In [ ]:
w = compute_w_log(hmm_7_state, x_short)
opt_path_prob_log(w)

Now do the same for `x_long`. What happens?

In [ ]:
# Your code here ...

### Obtaining an optimal path through backtracking

In [ ]:
def backtrack_log(model, x, w):
    pass

In [ ]:
w = compute_w_log(hmm_7_state, x_short)
z_viterbi_log = backtrack_log(hmm_7_state, x_short, w)

Now do the same for `x_long`. What happens?

In [ ]:
# Your code here ...

### Does it work?

Think about how to verify that your implementations of Viterbi (i.e. `compute_w`, `opt_path_prob`, `backtrack`, and there log-transformed variants `compute_w_log`, `opt_path_prob_log`, `backtrack_log`) are correct.

One thing that should hold is that the probability of a most likely path as computed by `opt_path_prob` (or `opt_path_prob_log`) for a given sequence of observables (e.g. `x_short` or `x_long`) should be equal to the joint probability of a corersponding most probable path as found by `backtrack` (or `backtrack_log`) and the given sequence of observables. Why?

Make an experiment that validates that this is the case for your implementations of Viterbi and `x_short` and `x_long`.

In [ ]:
# To access joint_prob and joint_prob_log, you must copy your implementations from last week here ...

def joint_prob(model, x, z):
    pass

def joint_prob_log(model, x, z):
    pass

# Check that opt_path_prob is equal to joint_prob(hmm_7_state, x_short, z_viterbi)

# Your code here ...

# Check that opt_path_prob_log is equal to joint_prob_log(hmm_7_state, x_short, z_viterbi_log)

# Your code here ...

# Do the above checks for x_long ...

# Your code here ...

Do your implementations pass the above checks?

### Does log-transformation matter?

Make an experiment that investigates how long the input string can be before `backtrack` and `backtrack_log` start to disagree on a most likely path and its probability.

In [ ]:
# Your code here ...

**Your answer here:**

For the 7-state model, `backtrack` and `backtrack_log` start to disagree on a most likely path and its probability for **i = ?** .